In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import re


# Read in all three datasets

In [2]:
books=pd.read_csv('books_out.csv')
borrowers=pd.read_csv('borrowers_out.csv')
rentals=pd.read_csv('rentals_out.csv')

# Examine books

Observations

* publication date is an object
* there are no NaN values :)

In [3]:
print(books.head())
print(books.describe())
print(books.info())
print(books.shape)


              UID           author  length publication_date  rental_period  \
0  81490378501273    Leslie Godwin     292       1985-11-30              7   
1   2265183237865  Theresa Scruggs      51       1922-02-15             30   
2  38355930389954    Elaine Turner      51       1990-02-27             30   
3  12676076159906  Claretta Dewitt     308       1988-06-23              7   
4  17366175799537   Douglas Barrio     243       1961-01-29             30   

            subject                                             title  
0         Religious  wispy Hiawatha Midwestern prolix sanitarium aloe  
1             SciFi                deterrent someday indent decennial  
2  Religious Novels          aquatic certificate flatus flowery slack  
3        HISTORICAL              pre scald pillage minor incomputable  
4           Fantasy                         hateful division Malagasy  
                UID       length  rental_period
count  2.999000e+03  2999.000000    2999.000000
mea

# Examine borrowers

Observations

* join date, date is an object
* there are no NaN values

In [4]:
print(borrowers.head())
print(borrowers.describe())
print(borrowers.info())
print(borrowers.shape)
print(borrowers.columns)

          DoB             UID address ethnic_group gender   join_date
0  1964-05-26   6407399394844    M1 2            D      M  1981-08-20
1  1935-10-19  76509601043881   M24 4            D      M  1973-10-14
2  1937-03-01   6792108939008   M33 5            D      M  1974-02-25
3  1964-06-18  10123457846743    M5 4            F      M  1973-01-07
4  1968-10-24  28422585028351   M40 7            D      M  1984-10-18
                UID
count  2.499000e+03
mean   5.028996e+13
std    2.891226e+13
min    3.569022e+09
25%    2.527671e+13
50%    5.067687e+13
75%    7.545922e+13
max    9.996030e+13
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 6 columns):
DoB             2499 non-null object
UID             2499 non-null int64
address         2499 non-null object
ethnic_group    2499 non-null object
gender          2499 non-null object
join_date       2499 non-null object
dtypes: int64(1), object(5)
memory usage: 117.2+ KB
None
(2499, 6)
Index(

# Examine rentals

Observations

* join date, date is an object
* there are no NaN values

In [5]:
print(rentals.head())
print(rentals.describe())
print(rentals.info())
print(rentals.shape)
print(rentals.columns)

              UID            book       borrower         out    returned
0  67337524405974  61277990803581  6407399394844  2003-02-09  2003-03-01
1  86074485118530  63129330503483  6407399394844  1994-05-31  1994-06-24
2  83474086825147  14481138517750  6407399394844  2008-06-04  2008-07-01
3  48570202472671  31927003009928  6407399394844  1998-04-09  1998-04-18
4  36296116903150  19608225871832  6407399394844  2003-07-24  2003-07-31
                UID          book      borrower
count  8.553160e+05  8.553160e+05  8.553160e+05
mean   4.999398e+13  5.093816e+13  4.991703e+13
std    2.888149e+13  2.875983e+13  2.886223e+13
min    1.657278e+07  5.453121e+10  3.569022e+09
25%    2.498286e+13  2.605777e+13  2.478789e+13
50%    4.995175e+13  5.172893e+13  5.023728e+13
75%    7.501783e+13  7.589374e+13  7.525122e+13
max    9.999990e+13  9.996550e+13  9.996030e+13
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855316 entries, 0 to 855315
Data columns (total 5 columns):
UID         855316 n

# Clean Column Names

* column names are inconsistent accross tables

In [6]:
rentals.columns=['uid_rental','uid_book','uid_borrowers','out','returned']
print(rentals.columns)


Index(['uid_rental', 'uid_book', 'uid_borrowers', 'out', 'returned'], dtype='object')


In [7]:
print(books.columns)
books.columns=['uid_book','author','length','publication_date','rental_period','subject','title']
print(books.columns)

Index(['UID', 'author', 'length', 'publication_date', 'rental_period',
       'subject', 'title'],
      dtype='object')
Index(['uid_book', 'author', 'length', 'publication_date', 'rental_period',
       'subject', 'title'],
      dtype='object')


In [8]:
print(borrowers.columns)
borrowers.columns=['dob','uid_borrowers','address','ethnic_group','gender','join_date']
print(borrowers.columns)

Index(['DoB', 'UID', 'address', 'ethnic_group', 'gender', 'join_date'], dtype='object')
Index(['dob', 'uid_borrowers', 'address', 'ethnic_group', 'gender',
       'join_date'],
      dtype='object')


# Merging Dataframes (Combine all)

1. firstly, lets merge books and rentals

In [9]:
books_rentals = pd.merge(left= rentals, right=books, 
                                left_on='uid_book', right_on='uid_book')

#books_rentals.to_csv('books_rentals.csv') <--did this to take a look in excel

2.Merge books_rentals and borrowers dataframes

In [10]:
combined = pd.merge(left= books_rentals, right=borrowers, 
                                left_on='uid_borrowers', right_on='uid_borrowers')
#combined.to_csv('combined.csv')


Just in case the combined dataframe is not easy to work with, lets merge borrowers and rentals

In [11]:
borrowers_rentals = pd.merge(left= rentals, right=borrowers, 
                            left_on='uid_borrowers', right_on='uid_borrowers')

# Create Late column (in 'combine' dataframe)
1. Convert 'returned' column from object to datetime
2. create days_late columns
3. create late columns
    * if days late is greater than rental period, then late
    
Note: 
* Late is True
* On Time is False

In [12]:
combined.returned=pd.to_datetime(combined.returned)
combined.out=pd.to_datetime(combined.out)
combined['days_late']=combined.returned-combined.out

print(combined.days_late.head())

0   20 days
1   24 days
2   27 days
3   22 days
4    9 days
Name: days_late, dtype: timedelta64[ns]


Note: 

* We must now make days_late the same dtype as rental_period

In [13]:
combined.days_late=combined.days_late.astype(dtype='int64') #this time in nanoseconds
combined.days_late=combined.days_late/86400000000000 #there are 8.64e+13 nanoseconds in a day (there must be a better way of doing this)
print(combined.days_late.head())

0    20.0
1    24.0
2    27.0
3    22.0
4     9.0
Name: days_late, dtype: float64


Lets create a new columns which returns True or False based on whether the data_late is greater than the rental period

In [14]:
#combined['Late']=combined.days_late(lambda x, y: func)

combined['late']= combined.days_late>=combined.rental_period
print(combined.late.head())

#False if the book is returned on time
#True if the book is returned late

0    False
1    False
2    False
3    False
4     True
Name: late, dtype: bool


In [15]:
print(combined.head()) #looking at the late column, and spot checking the csv file it seems to have done the trick.

       uid_rental        uid_book  uid_borrowers        out   returned  \
0  67337524405974  61277990803581  6407399394844 2003-02-09 2003-03-01   
1  86074485118530  63129330503483  6407399394844 1994-05-31 1994-06-24   
2  83474086825147  14481138517750  6407399394844 2008-06-04 2008-07-01   
3  74692526227755  14481138517750  6407399394844 2008-03-21 2008-04-12   
4  48570202472671  31927003009928  6407399394844 1998-04-09 1998-04-18   

              author  length publication_date  rental_period        subject  \
0        Donna Adell     343       1979-02-19             30     Historical   
1      Steven Briney     235       1980-03-22             30        fantasy   
2      Lessie Arnold     364       1926-11-28             30        fantasy   
3      Lessie Arnold     364       1926-11-28             30        fantasy   
4  Jerri Vandenbosch     232       1966-02-19              7  SHORT STORIES   

                                            title         dob address  \
0     L

# Create age column (from date of birth) 

(in combined dataframe)

In [16]:
combined.dob.dtypes #here we discover dob is an object
combined.dob.head()

0    1964-05-26
1    1964-05-26
2    1964-05-26
3    1964-05-26
4    1964-05-26
Name: dob, dtype: object

In [17]:
combined.dob=pd.to_datetime(combined.dob)
combined.dob.head()

0   1964-05-26
1   1964-05-26
2   1964-05-26
3   1964-05-26
4   1964-05-26
Name: dob, dtype: datetime64[ns]

In [18]:
from datetime import datetime
from datetime import date

def calculate_age(born):
    #born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

combined['age'] = combined.dob.apply(calculate_age)

combined.age.head()


0    54
1    54
2    54
3    54
4    54
Name: age, dtype: int64

In [19]:
combined.to_csv('combined.csv') 

# create age and late column in borrowers_rentals data frame

    so it turns out combinig all the dataframes made analysing the borrower table more difficult

In [20]:

#creates new late column 
borrowers_rentals.returned=pd.to_datetime(combined.returned)
borrowers_rentals.out=pd.to_datetime(combined.out)
borrowers_rentals['days_late']=combined.returned-combined.out
borrowers_rentals['late']= combined.days_late>=combined.rental_period


borrowers_rentals.dob=pd.to_datetime(borrowers_rentals.dob)
from datetime import datetime
from datetime import date

def calculate_age(born):
    #born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

borrowers_rentals['age'] = borrowers_rentals.dob.apply(calculate_age)


borrowers_rentals.head()

,uid_rental,uid_book,uid_borrowers,out,returned,dob,address,ethnic_group,gender,join_date,days_late,late,age
0,67337524405974,61277990803581,6407399394844,2003-02-09,2003-03-01,1964-05-26,M1 2,D,M,1981-08-20,20 days,False,54
1,86074485118530,63129330503483,6407399394844,1994-05-31,1994-06-24,1964-05-26,M1 2,D,M,1981-08-20,24 days,False,54
2,83474086825147,14481138517750,6407399394844,2008-06-04,2008-07-01,1964-05-26,M1 2,D,M,1981-08-20,27 days,False,54
3,48570202472671,31927003009928,6407399394844,2008-03-21,2008-04-12,1964-05-26,M1 2,D,M,1981-08-20,22 days,False,54
4,36296116903150,19608225871832,6407399394844,1998-04-09,1998-04-18,1964-05-26,M1 2,D,M,1981-08-20,9 days,True,54
